In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input

# Directory paths for real and deepfake images
train_dir = 'C:/Users/ksaks/Downloads/DATA/Training/train'  # Directory containing 'real' and 'deepfake' folders
val_dir = r'C:\Users\ksaks\Downloads\DATA\Training\VAL'  # Directory containing 'real' and 'deepfake' folders

# Set up image data generators
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Rescale image pixel values to [0, 1]
    rotation_range=20,  # Rotate images randomly within a range
    width_shift_range=0.2,  # Randomly shift images horizontally
    height_shift_range=0.2,  # Randomly shift images vertically
    shear_range=0.2,  # Randomly apply shearing transformations
    zoom_range=0.2,  # Randomly zoom in or out on the images
    horizontal_flip=True,  # Randomly flip the images horizontally
    fill_mode='nearest'  # Fill in missing pixels after transformation
)

val_datagen = ImageDataGenerator(rescale=1.0/255.0)  # Validation data is just rescaled

# Load images from directories and apply the above transformations
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Resize images to 224x224
    batch_size=32,  # Batch size
    class_mode='binary'  # Binary classification (real vs deepfake)
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),  # Resize images to 224x224
    batch_size=32,  # Batch size
    class_mode='binary'  # Binary classification (real vs deepfake)
)



Found 4812 images belonging to 2 classes.
Found 2068 images belonging to 2 classes.


In [2]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import TensorBoard
import os

# Load the VGG16 model with pre-trained weights (ImageNet)
# We exclude the top layers because we will add our own output layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


# Add custom layers on top of VGG16 for our classification task
model = models.Sequential([
    base_model,  # Add the VGG16 base model
    layers.Flatten(),  # Flatten the output from the base model
    layers.Dense(256, activation='relu'),  # Add a dense layer with ReLU activation
    layers.Dense(1, activation='sigmoid')  # Output layer with sigmoid activation (binary classification)
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model architecture
model.summary()





Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 21137729 (80.63 MB)
Trainable params: 6423041 (24.50 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [3]:
import os
from tensorflow.keras.callbacks import Callback

class TopKModelSaver(Callback):
    def __init__(self, directory, monitor='val_loss', k=5, mode='min'):
        super(TopKModelSaver, self).__init__()
        self.directory = directory
        self.monitor = monitor
        self.k = k
        self.mode = mode
        self.top_k_models = []  # List to store top k models' metadata

        # Ensure the directory exists
        os.makedirs(self.directory, exist_ok=True)

    def on_epoch_end(self, epoch, logs=None):
        current_value = logs.get(self.monitor)
        if current_value is None:
            print(f"Warning: Monitor value '{self.monitor}' is missing in logs.")
            return

        # Determine whether to add the current model
        should_save = False
        if len(self.top_k_models) < self.k:
            should_save = True
        else:
            if self.mode == 'min' and current_value < max(self.top_k_models, key=lambda x: x['value'])['value']:
                should_save = True
            elif self.mode == 'max' and current_value > min(self.top_k_models, key=lambda x: x['value'])['value']:
                should_save = True

        if should_save:
            # Construct the file path
            model_path = os.path.join(self.directory, f'model_epoch_{epoch + 1:02d}-{self.monitor}_{current_value:.4f}.h5')

            # Save the current model
            self.model.save(model_path)

            # Update the top_k_models list
            self.top_k_models.append({'value': current_value, 'path': model_path})
            self.top_k_models = sorted(self.top_k_models, key=lambda x: x['value'], reverse=(self.mode == 'max'))

            # If more than k models, remove the worst
            if len(self.top_k_models) > self.k:
                worst_model = self.top_k_models.pop()
                os.remove(worst_model['path'])  # Delete the file

            print(f"Model saved to {model_path}. Current top {self.k} models:")
            for model_info in self.top_k_models:
                print(f"  - {model_info['path']} (val: {model_info['value']:.4f})")

In [4]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

# Directory to store TensorBoard logs
log_dir = r'C:\Users\ksaks\Downloads\DATA'
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# # Early stopping to monitor validation loss and stop training if it doesn't improve
# early_stopping = EarlyStopping(
#     monitor='val_loss',  # Monitor validation loss
#     patience=5,          # Number of epochs with no improvement after which training will be stopped
#     verbose=1,           # Print updates
#     restore_best_weights=True  # Restore the best weights at the end of training
# )

# Create the TopKModelSaver callback
top_k_saver = TopKModelSaver(
    directory=r'C:\Users\ksaks\Downloads\DATA\Models',
    monitor='val_loss',
    k=20,
    mode='min'  # Change to 'max' if you want to maximize the monitored metric
)

# Train the model with the new callback
history = model.fit(
    train_generator,
    epochs=50,  # Number of epochs
    validation_data=val_generator,
    callbacks=[tensorboard_callback, top_k_saver]  # Include the new callback
)

Epoch 1/50


151/151 [==============================] - ETA: 0s - loss: 0.5541 - accuracy: 0.7943Model saved to C:\Users\ksaks\Downloads\DATA\Models\model_epoch_01-val_loss_0.8150.h5. Current top 20 models:
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_01-val_loss_0.8150.h5 (val: 0.8150)
151/151 [==============================] - 379s 2s/step - loss: 0.5541 - accuracy: 0.7943 - val_loss: 0.8150 - val_accuracy: 0.7205
Epoch 2/50


C:\Users\ksaks\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


151/151 [==============================] - ETA: 0s - loss: 0.3445 - accuracy: 0.8697Model saved to C:\Users\ksaks\Downloads\DATA\Models\model_epoch_02-val_loss_0.5823.h5. Current top 20 models:
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_02-val_loss_0.5823.h5 (val: 0.5823)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_01-val_loss_0.8150.h5 (val: 0.8150)
151/151 [==============================] - 378s 3s/step - loss: 0.3445 - accuracy: 0.8697 - val_loss: 0.5823 - val_accuracy: 0.7655
Epoch 3/50
151/151 [==============================] - ETA: 0s - loss: 0.3042 - accuracy: 0.8797Model saved to C:\Users\ksaks\Downloads\DATA\Models\model_epoch_03-val_loss_0.5981.h5. Current top 20 models:
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_02-val_loss_0.5823.h5 (val: 0.5823)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_03-val_loss_0.5981.h5 (val: 0.5981)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_01-val_loss_0.8150.h5 (val: 0.8150)
151/151 [===============

Epoch 12/50
151/151 [==============================] - ETA: 0s - loss: 0.2358 - accuracy: 0.8990Model saved to C:\Users\ksaks\Downloads\DATA\Models\model_epoch_12-val_loss_0.5177.h5. Current top 20 models:
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_12-val_loss_0.5177.h5 (val: 0.5177)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_08-val_loss_0.5299.h5 (val: 0.5299)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_09-val_loss_0.5340.h5 (val: 0.5340)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_11-val_loss_0.5401.h5 (val: 0.5401)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_10-val_loss_0.5617.h5 (val: 0.5617)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_07-val_loss_0.5753.h5 (val: 0.5753)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_06-val_loss_0.5797.h5 (val: 0.5797)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_02-val_loss_0.5823.h5 (val: 0.5823)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_03-val_loss_0.5981.h5 (val: 0

Epoch 18/50
151/151 [==============================] - ETA: 0s - loss: 0.2203 - accuracy: 0.9121Model saved to C:\Users\ksaks\Downloads\DATA\Models\model_epoch_18-val_loss_0.5466.h5. Current top 20 models:
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_17-val_loss_0.5111.h5 (val: 0.5111)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_12-val_loss_0.5177.h5 (val: 0.5177)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_08-val_loss_0.5299.h5 (val: 0.5299)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_09-val_loss_0.5340.h5 (val: 0.5340)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_11-val_loss_0.5401.h5 (val: 0.5401)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_18-val_loss_0.5466.h5 (val: 0.5466)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_14-val_loss_0.5501.h5 (val: 0.5501)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_10-val_loss_0.5617.h5 (val: 0.5617)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_16-val_loss_0.5677.h5 (val: 0

Epoch 22/50
151/151 [==============================] - ETA: 0s - loss: 0.2072 - accuracy: 0.9167Model saved to C:\Users\ksaks\Downloads\DATA\Models\model_epoch_22-val_loss_0.5636.h5. Current top 20 models:
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_17-val_loss_0.5111.h5 (val: 0.5111)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_12-val_loss_0.5177.h5 (val: 0.5177)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_19-val_loss_0.5229.h5 (val: 0.5229)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_08-val_loss_0.5299.h5 (val: 0.5299)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_09-val_loss_0.5340.h5 (val: 0.5340)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_21-val_loss_0.5374.h5 (val: 0.5374)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_11-val_loss_0.5401.h5 (val: 0.5401)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_18-val_loss_0.5466.h5 (val: 0.5466)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_14-val_loss_0.5501.h5 (val: 0

Epoch 26/50
151/151 [==============================] - ETA: 0s - loss: 0.2032 - accuracy: 0.9163Model saved to C:\Users\ksaks\Downloads\DATA\Models\model_epoch_26-val_loss_0.5628.h5. Current top 20 models:
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_17-val_loss_0.5111.h5 (val: 0.5111)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_12-val_loss_0.5177.h5 (val: 0.5177)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_23-val_loss_0.5223.h5 (val: 0.5223)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_19-val_loss_0.5229.h5 (val: 0.5229)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_08-val_loss_0.5299.h5 (val: 0.5299)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_09-val_loss_0.5340.h5 (val: 0.5340)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_21-val_loss_0.5374.h5 (val: 0.5374)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_25-val_loss_0.5396.h5 (val: 0.5396)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_11-val_loss_0.5401.h5 (val: 0

Epoch 31/50
151/151 [==============================] - ETA: 0s - loss: 0.1937 - accuracy: 0.9165Model saved to C:\Users\ksaks\Downloads\DATA\Models\model_epoch_31-val_loss_0.5359.h5. Current top 20 models:
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_29-val_loss_0.4869.h5 (val: 0.4869)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_17-val_loss_0.5111.h5 (val: 0.5111)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_30-val_loss_0.5145.h5 (val: 0.5145)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_12-val_loss_0.5177.h5 (val: 0.5177)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_23-val_loss_0.5223.h5 (val: 0.5223)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_19-val_loss_0.5229.h5 (val: 0.5229)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_08-val_loss_0.5299.h5 (val: 0.5299)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_28-val_loss_0.5337.h5 (val: 0.5337)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_09-val_loss_0.5340.h5 (val: 0

Epoch 40/50
151/151 [==============================] - ETA: 0s - loss: 0.1937 - accuracy: 0.9167Model saved to C:\Users\ksaks\Downloads\DATA\Models\model_epoch_40-val_loss_0.5369.h5. Current top 20 models:
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_29-val_loss_0.4869.h5 (val: 0.4869)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_39-val_loss_0.4940.h5 (val: 0.4940)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_17-val_loss_0.5111.h5 (val: 0.5111)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_30-val_loss_0.5145.h5 (val: 0.5145)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_12-val_loss_0.5177.h5 (val: 0.5177)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_23-val_loss_0.5223.h5 (val: 0.5223)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_19-val_loss_0.5229.h5 (val: 0.5229)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_08-val_loss_0.5299.h5 (val: 0.5299)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_28-val_loss_0.5337.h5 (val: 0

Epoch 50/50
151/151 [==============================] - ETA: 0s - loss: 0.1908 - accuracy: 0.9175Model saved to C:\Users\ksaks\Downloads\DATA\Models\model_epoch_50-val_loss_0.4687.h5. Current top 20 models:
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_50-val_loss_0.4687.h5 (val: 0.4687)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_49-val_loss_0.4737.h5 (val: 0.4737)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_29-val_loss_0.4869.h5 (val: 0.4869)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_39-val_loss_0.4940.h5 (val: 0.4940)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_48-val_loss_0.4950.h5 (val: 0.4950)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_17-val_loss_0.5111.h5 (val: 0.5111)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_30-val_loss_0.5145.h5 (val: 0.5145)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_12-val_loss_0.5177.h5 (val: 0.5177)
  - C:\Users\ksaks\Downloads\DATA\Models\model_epoch_23-val_loss_0.5223.h5 (val: 0

In [5]:
model.save('C:/Users/ksaks/Downloads/DATA/Models/lasr_model_VGG.h5')